In [ ]:
# estou adaptando esse notebook para armazenar os trades também

In [1]:
import Ativo as at
import FileManager as fman
import pickle
import pandas as pd
import numpy as np
import datetime

In [2]:
fm = fman.FileManager()

In [ ]:
# intersection of list
# https://www.geeksforgeeks.org/python-intersection-two-lists/


In [ ]:
names_pennystocks = fm.getNames()
names_free_float = fm.getFreeFloatNames()
# len(names) # 2948
# len(names_free_float) # 4041
names = list(set(names_pennystocks) & set(names_free_float))# faz a intersection das duas listas
names.sort() # sort é inline
# len(names_intersec) # 2682

In [ ]:
al = []
# for n in names[0:10]:
for n in names:
    print(n)
    try:
        a = at.Ativo(n, fm[n])
        for intraday in a.intraDays:
            d = {'name': a.name,
                 'date': intraday.date,
                 'freefloat': fm.getFreeFloat(n),
                 'stats': intraday.stats}
            al.append( d )
    except IndexError:
        print("Ativo sem nenhum dado ou dado inconsistente")

In [ ]:
# como temos um arquivo grande al, que é uma lista de dicts, é importante salvar em um file para que não precisemos
# rodar toda a análise novamente cada vez que queremos analisar ou filtrar os dias*ativos de uma maneira diferente
# https://stackabuse.com/reading-and-writing-lists-to-a-file-in-python/
# https://pythonspot.com/save-a-dictionary-to-a-file/
with open('AtivoDiaList2.pkl', 'wb') as filehandle: # w de write e b de binary
    pickle.dump(al,filehandle)

In [6]:
# e agora pra abrir o dictionary que foi salvo em um pickle?
with open('AtivoDiaList2.pkl', 'rb') as filehandle: # w de write e b de binary
    ativo_dia_list = pickle.load(filehandle) # notar como para salvar precisamos passar a list e o filehandle, mas pra abrir não

In [7]:
prevol_threshold = 8000000
open_dolar_threshold = 1
gap_threshold = 0.10
F_low_threshold = 1
F_high_threshold = 5

def make_filter_prevol(threshold):
     return lambda ad: ad['stats']['volPre'] >= threshold

def make_filter_open_dolar(threshold):
     return lambda ad: ad['stats']['openValue'] >= threshold

def make_filter_gap(threshold):
    return lambda ad: ad['stats']['gap'] >= threshold

def make_filter_F(low_threshold, high_threshold):
    return lambda ad: F_low_threshold <= ad['stats']['moneyVolPre']/ad['freefloat'] <= F_high_threshold

prevol_greater_than = make_filter_prevol(prevol_threshold)
open_greater_than_dolar = make_filter_open_dolar(open_dolar_threshold)
gap_greater_than = make_filter_gap(gap_threshold)
F_between = make_filter_F(F_low_threshold,F_high_threshold)

In [8]:
# FILTER 1 + FILTER 2 + FILTER 3 + FILTER 4
# as funções-filtro prevol_greater_than_800k(), open_greater_than_1dolar(), gap_greater_than_8percent() 
# factor_f_greater_than2 já foram definidas

filtered_ativo_dias =  filter(prevol_greater_than, ativo_dia_list)
filtered_ativo_dias =  filter(open_greater_than_dolar, filtered_ativo_dias)
filtered_ativo_dias =  filter(gap_greater_than, filtered_ativo_dias)
filtered_ativo_dias =  filter(F_between, filtered_ativo_dias)
filtered_ativo_dias = list(filtered_ativo_dias)

In [9]:
# vamos primeiramente criar um dataframe vazio, mas com as colunas bem definidas
# https://www.geeksforgeeks.org/python-pandas-dataframe-append/#:~:text=append()%20function%20is%20used,are%20populated%20with%20NaN%20value.&text=ignore_index%20%3A%20If%20True%2C%20do%20not%20use%20the%20index%20labels.
df = pd.DataFrame({'name':[],
                   'date':[],
                   'freefloat':[],
                   'volPre':[],
                   'gap':[],
                   'openToSpike%':[],
                   'minsToSpike':[],
                   'volToSpike':[],
                   'spikeToLow%':[],
                   'minsToLowAfterSpike':[],
                   'spikeToPreVolF':[],
                   'factorF':[]})
# agora popula esse dataframe com todos AtivosXDia que passaram nos critérios
for ad in filtered_ativo_dias:
    secondsToSpike = datetime.datetime.combine(datetime.date.today(), ad['stats']['highCoreTime']) - datetime.datetime.combine(datetime.date.today(), datetime.time(9,31))
    minutesToLowAfterSpike = datetime.datetime.combine(datetime.date.today(), ad['stats']['lowAfterHighTime']) - datetime.datetime.combine(datetime.date.today(), ad['stats']['highCoreTime'])

    df = df.append({'name':ad['name'],
                       'date':ad['date'].strftime("%d/%m/%Y"),
                       'freefloat':ad['freefloat'],
                       'volPre':ad['stats']['volPre'],
                       'gap':ad['stats']['gap'],
                       'openToSpike%':ad['stats']['openToSpikePercent'],
                       'minsToSpike':secondsToSpike.total_seconds()/60,
                       'volToSpike':ad['stats']['volumeToSpike'],
                       'spikeToLow%':ad['stats']['spikeToLowPercent'],
                       'minsToLowAfterSpike':minutesToLowAfterSpike.total_seconds()/60,
                       'spikeToPreVolF':ad['stats']['spikeToPreVolFactor'],
                       'factorF':ad['stats']['moneyVolPre']/ad['freefloat']},
                          ignore_index=True)

In [ ]:
short_after = 0.1
exit_target = 0.4
exit_stop = 0.2

trades = []
for ad in filtered_ativo_dias:
#    if "a" in locals(): # se a ja tiver sido instanciado
#        if a.name == ad['name']:
#            pass
    a = at.Ativo(ad['name'],fm[ad['name']])
    intra = a.fromDay(ad['date'])
    trades.append({'name': ad['name'],
                   'date': ad['date'],
                   'trade': intra.checkForTrade(short_after, exit_target, exit_stop)})

In [ ]:
start = 100000
n_trades = 0
for t in trades:
    if t['trade']: # se o trade não for None, lembrar que None é falsey
        start = start*(1+t['trade']['profit'])
        #print(start)
        n_trades += 1
print('End Money:', start)
print('Number of Trades:', n_trades)
print('Number of ativo-dias:', len(filtered_ativo_dias) )

In [ ]:
len(filtered_ativo_dias)

In [ ]:
ad = filtered_ativo_dias[150]
ad

In [ ]:
trades

In [ ]:
print('teste','222')

In [ ]:
trades

In [3]:
fm.getFreeFloatNames()

['AIXN',
 'VXEL',
 'WORX',
 'WOWMF',
 'AXREF',
 'ITXXF',
 'MMDDF',
 'ALID',
 'SNANF',
 'LDSN',
 'ELYS',
 'UCLE',
 'TFECY',
 'SUME',
 'JRVMF',
 'SPIN',
 'HEYBF',
 'VYCO',
 'TIXC',
 'AHNR',
 'WRHLF',
 'ABMT',
 'KNRLF',
 'CAPC',
 'APXT',
 'VGGOF',
 'ASDN',
 'ICOTF',
 'JG',
 'NRVTF',
 'FUSEF',
 'AMSLF',
 'LSMG',
 'WINRD',
 'MWXRF',
 'HQDA',
 'VIHD',
 'VTLR',
 'MGWFF',
 'MLGCF',
 'JPOTF',
 'STSBF',
 'MCURF',
 'GNAL',
 'BHNGF',
 'BRSF',
 'TOUR',
 'CTGL',
 'CBAT',
 'PCLOF',
 'INXSF',
 'IPDN',
 'ANFC',
 'CLOK',
 'SRNE',
 'PNDHF',
 'AAIRF',
 'WYY',
 'WSCRF',
 'TCLRY',
 'KMPH',
 'ACUR',
 'TTTSF',
 'USDR',
 'SFT',
 'GOLXF',
 'HLRTF',
 'VSTM',
 'LONCF',
 'NXEN',
 'NQMLF',
 'NVOS',
 'ZMWYF',
 'VIVXF',
 'ERYP',
 'OPESU',
 'MUNMF',
 'PIC',
 'ASPCF',
 'GBLTF',
 'VPHIF',
 'IDKFF',
 'STHI',
 'FNNZF',
 'VYGVF',
 'ETST',
 'GTHP',
 'THCB',
 'RMG.U',
 'GRCK',
 'FLURF',
 'CBDY',
 'ROIUF',
 'NAKD',
 'AMCT',
 'MOST',
 'BMNM',
 'TAOIF',
 'RMG',
 'EXDI',
 'NSCIF',
 'RWBYF',
 'SCIA',
 'EVLLF',
 'HOILF',
 'FTMDF',

In [5]:
fm.getFreeFloat('VTLR')

32049516

In [10]:
df

,name,date,freefloat,volPre,gap,openToSpike%,minsToSpike,volToSpike,spikeToLow%,minsToLowAfterSpike,spikeToPreVolF,factorF
0,ABUS,24/07/2020,61552279.0,15640564.0,0.289197,0.130326,3.0,13391918.0,-0.471175,362.0,0.856230,1.868844
1,AGE,08/10/2020,21152649.0,23685891.0,1.653714,0.754464,46.0,136519495.0,-0.432570,273.0,5.763747,2.103786
2,AIHS,09/07/2020,29245409.0,14186559.0,0.840000,0.021739,2.0,5413726.0,-0.391489,315.0,0.381610,1.120534
3,AMC,11/05/2020,55076405.0,11325108.0,0.560976,0.001562,0.0,3088550.0,-0.237129,67.0,0.272717,1.368429
4,AYRO,06/07/2020,21674593.0,12995554.0,0.546584,0.642570,112.0,117236498.0,-0.405868,219.0,9.021277,2.903757
5,BDR,19/08/2020,3386626.0,9016896.0,1.104986,1.375000,22.0,35081341.0,-0.655263,363.0,3.890623,4.190665
6,BGFV,09/07/2020,18784403.0,8170825.0,0.651042,0.059937,0.0,860331.0,-0.267857,125.0,0.105293,1.581584
7,BHTG,29/06/2020,18258772.0,15647587.0,1.395210,0.100000,0.0,3405029.0,-0.420455,271.0,0.217607,2.910233
8,BOXL,16/07/2020,43104679.0,12744220.0,0.409283,0.392216,37.0,65319609.0,-0.309677,247.0,5.125430,1.000192
9,BOXL,12/08/2020,43104679.0,16881394.0,0.286783,0.038760,3.0,5455545.0,-0.126866,21.0,0.323169,1.047995
